## 순환신경망 구현 및 학습

In [1]:
import time
import tensorflow as tf

## 하이퍼 파라미터

In [2]:
EPOCHS = 10
NUM_WORDS = 10000
MODEL = 1 # 1: RNN, 2:LSTM, 3:GRU

## 모델 정의

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        
        # 길이가 10000인 one-hot vector를 길이가 16인 feature vector로 변경함
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16)
        
        # 32 : RNN의 갯수
        if MODEL == 1:
            self.rnn = tf.keras.layers.SimpleRNN(32)
        elif MODEL == 2:
            self.rnn = tf.keras.layers.LSTM(32)
        else:
            self.rnn = tf.keras.layers.GRU(32)
        self.dense = tf.keras.layers.Dense(2, activation='softmax')
        
    def call(self, x, training=None, mask=None):
        x = self.emb(x)
        x = self.rnn(x)
        return self.dense(x)

## 학습, 테스트 루프 정의

In [4]:
# Implement training loop
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images, training=False)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 데이터셋 준비


In [11]:
imdb = tf.keras.datasets.imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=NUM_WORDS)

print(x_train[0:2])
print(y_train[0:2])

# 입력 문장의 길이가 각각 달라서 32로 조정하고
# 32보다 길면 뒤쪽부터 32개를 넣고,
# 32보다 작으면 앞쪽부터 padding을 0으로 채워줌
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=32)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test,
                                                      value=0,
                                                      padding='pre',
                                                      maxlen=32)

print(x_train.shape)
print(x_train[0:2])
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

[list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32])
 list([1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463, 

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [6]:
# Create model
model = MyModel()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 동작

In [7]:
if MODEL == 1:
    print('MODEL : RNN')
elif MODEL == 2:
    print('MODEL : LSTM')
else:
    print('MODEL : GRU')

# 시간측정 필요. 노트북의 경우 SimpleRNN 방식일 경우 1 epoch당 10초 정도 소요됨
# 시간측정 필요. 노트북의 경우 LSTM 방식일 경우 1 epoch당 18초 정도 소요됨
# 시간측정 필요. 노트북의 경우 GRU 방식일 경우 1 epoch당 15초 정도 소요됨
t = time.time()
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_seqs, test_labels in test_ds:
        test_step(model, test_seqs, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    ct = time.time()
    print('{} seconds elapsed.'.format(ct - t))
    t = ct    

MODEL : RNN
Epoch 1, Loss: 0.5839154124259949, Accuracy: 66.09600067138672, Test Loss: 0.4963189661502838, Test Accuracy: 75.9679946899414
14.715745687484741 seconds elapsed.
Epoch 2, Loss: 0.3629997670650482, Accuracy: 84.2040023803711, Test Loss: 0.49189624190330505, Test Accuracy: 77.2959976196289
10.089024305343628 seconds elapsed.
Epoch 3, Loss: 0.20407110452651978, Accuracy: 92.3239974975586, Test Loss: 0.6146169304847717, Test Accuracy: 75.91600036621094
10.112027406692505 seconds elapsed.
Epoch 4, Loss: 0.08782748132944107, Accuracy: 96.98799896240234, Test Loss: 0.8317991495132446, Test Accuracy: 73.02400207519531
9.963084697723389 seconds elapsed.
Epoch 5, Loss: 0.03520545735955238, Accuracy: 98.93599700927734, Test Loss: 1.0458087921142578, Test Accuracy: 73.68800354003906
9.964804649353027 seconds elapsed.
Epoch 6, Loss: 0.01623946987092495, Accuracy: 99.50799560546875, Test Loss: 1.2195740938186646, Test Accuracy: 75.1240005493164
9.938246250152588 seconds elapsed.
Epoch 7